In [2]:
import pandas as pd
from zipfile import Path as ZipPath
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
	
import matplotlib as mpl
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    "text.usetex": True,
    "font.family": "serif",
})
import matplotlib.pyplot as plt


# wczytywanie bezpośrednio z archiwum

In [3]:
fps = {x.stem: x for x in ZipPath('Dopasowanie.zip', at='/').iterdir()}
fps                                                                

{'IRL-2.2': Path('Dopasowanie.zip', 'IRL-2.2.csv'),
 'IRL-2.1': Path('Dopasowanie.zip', 'IRL-2.1.csv'),
 'trpp': Path('Dopasowanie.zip', 'trpp.csv')}

In [4]:
for k,v in fps.items():
    print(k, v.exists())

IRL-2.2 True
IRL-2.1 True
trpp True


In [5]:
%%time

dfs = {}
for k,v in fps.items():
    with v.open() as f:
        dfs[k] = pd.read_csv(f, parse_dates=['UTC'], index_col='UTC')


CPU times: user 1.41 s, sys: 64.8 ms, total: 1.48 s
Wall time: 1.49 s


# połączenie w jednego dataframe


In [6]:
df = pd.concat(
    [
        dfs["IRL-2.1"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.1"}),
        dfs["IRL-2.2"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.2"}),
        dfs["trpp"]
    ],
    axis=1
).sort_index()


In [7]:
print(df.columns.tolist())
print(dfs.keys())

['IRL-2.1', 'IRL-2.2', 'PLC_time', 'PAR', 'PK1', 'PK2', 'PK3', 'PK4', 'PK5', 'PK6', 'PB1', 'PB2', 'PB3', 'PB4', 'PB5', 'PB6']
dict_keys(['IRL-2.2', 'IRL-2.1', 'trpp'])


In [8]:
pk6 = dfs["trpp"]["PK6"].astype(float)

pk6 = pk6.loc[
    "2025-12-08 19:50:00" :
    "2025-12-08 21:30:00"
].dropna()



In [9]:
min_duration = pd.Timedelta(seconds=50)
tolerance = 1  # mm – dopuszczalna fluktuacja na plateau

segments = []
start_time = pk6.index[0]
ref_value = pk6.iloc[0]
values = [ref_value]

for t, v in pk6.iloc[1:].items():
    if abs(v - ref_value) <= tolerance:
        values.append(v)
    else:
        duration = t - start_time
        if duration >= min_duration:
            segments.append({
                "start": start_time,
                "end": t,
                "czas_trwania": duration.total_seconds(),
                "wartośc": np.mean(values),
                "mediana": np.median(values),
                "std": np.std(values),
                "n": len(values),
            })
        # reset
        start_time = t
        ref_value = v
        values = [v]

# ostatni segment
duration = pk6.index[-1] - start_time
if duration >= min_duration:
    segments.append({
        "start": start_time,
        "end": pk6.index[-1],
        "czas_trwania": duration.total_seconds(),
        "wartośc": np.mean(values),
        "mediana": np.median(values),
        "std": np.std(values),
        "n": len(values),
    })

steps = pd.DataFrame(segments)
steps.insert(0, "krok", np.arange(1, len(steps) + 1))


In [10]:
print(f"Liczba wykrytych schodków: {len(steps)}")
print(steps)

steps.to_csv("PK6_schodki_30s.csv", index=False)
# Wykres z zaznaczonymi schodkami
plt.figure(figsize=(10, 6))
plt.plot(pk6.index, pk6.values, label='PK6', color='blue')

Liczba wykrytych schodków: 12
    krok                      start                        end  czas_trwania  \
0      1 2025-12-08 19:50:00.423287 2025-12-08 19:56:33.093081    392.669794   
1      2 2025-12-08 19:56:54.845774 2025-12-08 20:08:24.881573    690.035799   
2      3 2025-12-08 20:08:46.642677 2025-12-08 20:17:00.894929    494.252252   
3      4 2025-12-08 20:17:09.182889 2025-12-08 20:24:58.534162    469.351273   
4      5 2025-12-08 20:25:03.722477 2025-12-08 20:32:41.706589    457.984112   
5      6 2025-12-08 20:32:48.954115 2025-12-08 20:40:56.886526    487.932411   
6      7 2025-12-08 20:41:03.103150 2025-12-08 20:49:02.831197    479.728047   
7      8 2025-12-08 20:49:10.082811 2025-12-08 20:56:22.094478    432.011667   
8      9 2025-12-08 20:56:30.385008 2025-12-08 21:02:34.016866    363.631858   
9     10 2025-12-08 21:02:44.378541 2025-12-08 21:09:11.917870    387.539329   
10    11 2025-12-08 21:09:21.245624 2025-12-08 21:14:32.032670    310.787046   
11    12 2

In [11]:
# niepewność średniej: u(x̄) = s / sqrt(n)
steps["niepewność pomiarowa"] = steps["std"] / np.sqrt(steps["n"])

table = steps[[
    "krok",
    "wartośc",
    "niepewność pomiarowa"
]].copy()

# usunięcie pierwszego (początkowego) poziomu
table = table.iloc[1:].reset_index(drop=True)

# nazwy kolumn po polsku
table = table.rename(columns={
    "krok": "Numer kroku pomiarowego",
    "wartośc": "Zagłębienie pręta PK6 [mm]",
    "niepewność pomiarowa": "Niepewność pomiarowa [mm]"
})

# poprawna numeracja kroków 1–11
table["Numer kroku pomiarowego"] = range(1, len(table) + 1)

# sensowne zaokrąglenie
table["Zagłębienie pręta PK6 [mm]"] = table["Zagłębienie pręta PK6 [mm]"].round(1)
table["Niepewność pomiarowa [mm]"] = table["Niepewność pomiarowa [mm]"].round(2)

print(table)


    Numer kroku pomiarowego  Zagłębienie pręta PK6 [mm]  \
0                         1                       219.0   
1                         2                       307.3   
2                         3                       388.9   
3                         4                       450.0   
4                         5                       517.6   
5                         6                       584.0   
6                         7                       646.0   
7                         8                       712.0   
8                         9                       790.0   
9                        10                       883.0   
10                       11                      1099.3   

    Niepewność pomiarowa [mm]  
0                        0.00  
1                        0.02  
2                        0.02  
3                        0.01  
4                        0.02  
5                        0.00  
6                        0.00  
7                        0.00  
8  

In [13]:
plateaus_table = table["Zagłębienie pręta PK6 [mm]"].to_numpy()
np.array([plateaus_table])
print(plateaus_table)


[ 219.   307.3  388.9  450.   517.6  584.   646.   712.   790.   883.
 1099.3]
